In [1]:
import pandas as pd

In [7]:
# Open csv as pandas from local data folder
url = 'test_set_with_label.csv'  # Update with your file's URL
df = pd.read_csv(url)

# Optional: Print some info to check
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 317 entries, 0 to 316
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Unnamed: 0.1        317 non-null    int64 
 1   Unnamed: 0          317 non-null    int64 
 2   Interview Question  317 non-null    object
 3   Interview Answer    317 non-null    object
 4   Question            317 non-null    object
 5   Annotator1          317 non-null    object
 6   Annotator2          317 non-null    object
 7   Annotator3          317 non-null    object
 8   Label               317 non-null    object
dtypes: int64(2), object(7)
memory usage: 22.4+ KB


In [8]:
df

,Unnamed: 0.1,Unnamed: 0,Interview Question,Interview Answer,Question,Annotator1,Annotator2,Annotator3,Label
0,0,0,"\nQ. What about the redline, sir?","\nThe President. Well, the world has made it c...",Inquiring about the status or information reg...,2.1 Dodging,2.4 General,2.1 Dodging,Indirect
1,1,1,\nQ. Will you invite them to the White House t...,\nPresident Obama. I think that anytime and an...,Will you invite them to the White House to neg...,2.2 Deflection,2.4 General,2.4 General,Indirect
2,2,2,"\nQ. Harsh. Mr. President, Japan has dropped t...",\nThe President. I think that the purpose of t...,Why was it necessary for Japan to drop the thr...,1.1 Explicit,1.2 Implicit,1.2 Implicit,Indirect
3,3,3,\nQ. The Lebanese Prime Minister is demanding ...,\nThe President. I'll let Condi talk about the...,When will we see this resolution?,1.1 Explicit,2.4 General,2.4 General,Indirect
4,4,4,"\nQ. Thank you, Mr. President. Back on Iraq, a...","\nThe President. No, I don't consider it a cre...",Updating the figure of Iraqi deaths,2.1 Dodging,1.2 Implicit,2.1 Dodging,Indirect
...,...,...,...,...,...,...,...,...,...
312,312,312,"\nQ. If so, why? And do you believe that the b...",\nThe President. I believe that the situation ...,"If so, why?",1.2 Implicit,2.1 Dodging,1.2 Implicit,Indirect
313,313,313,"\nQ. Yes, indeed. In reading the 1559 resoluti...","\nThe President. Well, the people who should g...",Request for the speaker's understanding and re...,2.2 Deflection,2.2 Deflection,2.2 Deflection,Indirect
314,314,314,"\nQ. Mr. President, we know that you talked ab...",\nPresident Bush. We strategized on both issue...,"Mr. President, we know that you talked about I...",1.1 Explicit,1.1 Explicit,1.1 Explicit,Direct Reply
315,315,315,\nQ. Is there anything that's disappointed you...,\nThe President. I was concerned at first abou...,Is there anything that's disappointed you abo...,2.2 Deflection,1.2 Implicit,1.2 Implicit,Indirect
